## 联合微调

In [1]:
from keras.applications import *
from keras.preprocessing.image import *
import h5py
import math
import gc
from keras.models import *
from keras.layers import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from keras.callbacks import *
import tensorflow as tf
import sys
sys.path.append("D:\\PyCharm\\PyProjects\\NN\\")
from nets.DenseNet.densenet161 import DenseNet161
from nets.resnet152 import ResNet152
import warnings
warnings.filterwarnings('ignore')

def d_preprocess_input(x):
    x = x[:, :, ::-1]
    x[:, :, 0] = (x[:, :, 0] - 103.94) * 0.017
    x[:, :, 1] = (x[:, :, 1] - 116.78) * 0.017
    x[:, :, 2] = (x[:, :, 2] - 123.68) * 0.017
    return x
def r_preprocess_input(x):
    # 'RGB'->'BGR'
    x = x[:, :, ::-1]
    # Zero-center by mean pixel
    x[:, :, 0] -= 103.939
    x[:, :, 1] -= 116.779
    x[:, :, 2] -= 123.68
    return x

batch_size=32
ft_epoch=8
r_image_size=(224,224)
d_image_size=(224,224)
x_image_size=(299,299)
r_freeze_layer=436
d_freeze_layer=408
x_freeze_layer=66
train_dir='bddogtrain\\data'
top_weights_path='weights\\top_+weights.h5'
d_weights_path = 'nets\\DenseNet\\imagenet_models\\densenet161_weights_tf.h5'

r_input_tensor = Input((224, 224, 3),name='r_input')
r_model = ResNet152(input_tensor=r_input_tensor, weights='imagenet', include_top=False)
r_ = GlobalAveragePooling2D()(r_model.output)

d_model = DenseNet161(reduction=0.5,weights_path=d_weights_path)
d_model.layers[0].name='d_input'
d_input_tensor=d_model.input
d_ = d_model.output

x_input_tensor = Input((299, 299, 3),name='x_input')
x_model = Xception(input_tensor=x_input_tensor, weights='imagenet', include_top=False)
x_ = GlobalAveragePooling2D()(x_model.output)

print(r_.shape,d_.shape,x_.shape)
cc = concatenate([r_, d_, x_],axis=-1)

top_model_input=Input((int(cc.shape[-1]),))
x = Dropout(0.5)(top_model_input)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
prediction = Dense(100, activation='softmax')(x)
top_model=Model(top_model_input,prediction)
top_model.load_weights(top_weights_path)#在预训练好顶层模型基础上微调

output_tensor = top_model(cc)
for layer in r_model.layers:
    layer.name='r_'+layer.name
for layer in d_model.layers:
    layer.name='d_'+layer.name
for layer in x_model.layers:
    layer.name='x_'+layer.name
model = Model(inputs=[r_input_tensor,d_input_tensor,x_input_tensor], outputs=output_tensor)
# model.layers[-1].name='output'

for layer in r_model.layers[:r_freeze_layer]:
    layer.trainable = False
for layer in r_model.layers[r_freeze_layer:]:
    layer.trainable = True
for layer in d_model.layers[:d_freeze_layer]:
    layer.trainable = False
for layer in d_model.layers[d_freeze_layer:]:
    layer.trainable = True
for layer in x_model.layers[:x_freeze_layer]:
    layer.trainable = False
for layer in x_model.layers[x_freeze_layer:]:
    layer.trainable = True

model.compile(optimizer=optimizers.SGD(lr=0.0001, momentum=0.9),loss='categorical_crossentropy',metrics=['accuracy'])

r_gen = ImageDataGenerator(
    preprocessing_function=r_preprocess_input
)
d_gen = ImageDataGenerator(
    preprocessing_function=d_preprocess_input
)
x_gen = ImageDataGenerator(
    preprocessing_function=xception.preprocess_input
)

classes=list(range(100))
for i,c in zip(range(100),classes):
    classes[i]=str(c)

r_generator = r_gen.flow_from_directory(
    train_dir,
    target_size=r_image_size,
    batch_size=batch_size,
    # shuffle=False,
    seed=1000,
    classes=classes
)
d_generator = d_gen.flow_from_directory(
    train_dir,
    target_size=d_image_size,
    batch_size=batch_size,
    # shuffle=False,
    seed=1000,
    classes=classes
)
x_generator = x_gen.flow_from_directory(
    train_dir,
    target_size=x_image_size,
    batch_size=batch_size,
    # shuffle=False,
    seed=1000,
    classes=classes
)

def mygen():
    while True:
        r=next(r_generator)
        d=next(d_generator)
        x=next(x_generator)
        x1=r[0]
        x2=d[0]
        x3=x[0]
        y=r[1]
        yield ({'r_r_input': x1, 'd_d_input': x2, 'x_x_input': x3}, y)

three_gen=mygen()
model.fit_generator(
    three_gen,
    steps_per_epoch= math.ceil(len(x_generator.filenames)/batch_size),
    epochs=ft_epoch,

)
gc.collect()


Using TensorFlow backend.


(?, 2048) (?, 2208) (?, 2048)
Found 37489 images belonging to 100 classes.
Found 37489 images belonging to 100 classes.
Found 37489 images belonging to 100 classes.


0

In [2]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

## Prediction

In [ ]:
from keras.applications import *
from keras.preprocessing.image import *
import h5py
import math
import gc
from keras.models import *
from keras.layers import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from keras.callbacks import *
import tensorflow as tf
import sys
sys.path.append("D:\\PyCharm\\PyProjects\\NN\\")
from nets.DenseNet.densenet161 import DenseNet161
from nets.resnet152 import ResNet152
import warnings
warnings.filterwarnings('ignore')

def d_preprocess_input(x):
    x = x[:, :, ::-1]
    x[:, :, 0] = (x[:, :, 0] - 103.94) * 0.017
    x[:, :, 1] = (x[:, :, 1] - 116.78) * 0.017
    x[:, :, 2] = (x[:, :, 2] - 123.68) * 0.017
    return x
def r_preprocess_input(x):
    # 'RGB'->'BGR'
    x = x[:, :, ::-1]
    # Zero-center by mean pixel
    x[:, :, 0] -= 103.939
    x[:, :, 1] -= 116.779
    x[:, :, 2] -= 123.68
    return x
batch_size=10
r_image_size=(224,224)
d_image_size=(224,224)
x_image_size=(299,299)
test_dir='bddogtrain\\finaltestcrop'
ft_weights_file='weights\\ft_weights.h5'
d_weights_path = 'nets\\DenseNet\\imagenet_models\\densenet161_weights_tf.h5'

r_input_tensor = Input((224, 224, 3),name='r_input')
r_model = ResNet152(input_tensor=r_input_tensor, weights='imagenet', include_top=False)
r_ = GlobalAveragePooling2D()(r_model.output)

d_model = DenseNet161(reduction=0.5,weights_path=d_weights_path)
d_model.layers[0].name='d_input'
d_input_tensor=d_model.input
d_ = d_model.output

x_input_tensor = Input((299, 299, 3),name='x_input')
x_model = Xception(input_tensor=x_input_tensor, weights='imagenet', include_top=False)
x_ = GlobalAveragePooling2D()(x_model.output)

print(r_.shape,d_.shape,x_.shape)
cc = concatenate([r_, d_, x_],axis=-1)

top_model_input=Input((int(cc.shape[-1]),))
x = Dropout(0.5)(top_model_input)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
prediction = Dense(97, activation='softmax')(x)
top_model=Model(top_model_input,prediction)

output_tensor = top_model(cc)
for layer in r_model.layers:
    layer.name='r_'+layer.name
for layer in d_model.layers:
    layer.name='d_'+layer.name
for layer in x_model.layers:
    layer.name='x_'+layer.name
model = Model(inputs=[r_input_tensor,d_input_tensor,x_input_tensor], outputs=output_tensor)

model.load_weights(ft_weights_file)

r_gen = ImageDataGenerator(
    preprocessing_function=r_preprocess_input
)
d_gen = ImageDataGenerator(
    preprocessing_function=d_preprocess_input
)
x_gen = ImageDataGenerator(
    preprocessing_function=xception.preprocess_input
)

r_generator = r_gen.flow_from_directory(
    test_dir,
    batch_size=batch_size,
    target_size=r_image_size,
    shuffle=False,
    class_mode=None
)
d_generator = d_gen.flow_from_directory(
    test_dir,
    batch_size=batch_size,
    target_size=d_image_size,
    shuffle=False,
    class_mode=None
)
x_generator = x_gen.flow_from_directory(
    test_dir,
    batch_size=batch_size,
    target_size=x_image_size,
    shuffle=False,
    class_mode=None
)

def mygen():
    while True:
        x1=next(r_generator)
        x2=next(d_generator)
        x3=next(x_generator)
        yield {'r_r_input': x1, 'd_d_input': x2, 'x_x_input': x3}

three_gen=mygen()
y_pred=model.predict_generator(three_gen,math.ceil(len(x_generator.filenames)/batch_size),verbose=1)

Using TensorFlow backend.


In [2]:
y_pred=np.argmax(y_pred,axis=1)

In [3]:
y_pred

array([36, 71, 27, ..., 80, 23, 50], dtype=int64)

In [4]:
df_loc = pd.read_csv("D:\\PyCharm\\PyProjects\\NN\\submission.csv")
df_loc['class']=y_pred
dic={0: 0,
 1: 1,
 2: 2,
 3: 3,
 4: 4,
 5: 5,
 6: 6,
 7: 7,
 8: 8,
 9: 9,
 10: 10,
 11: 11,
 12: 12,
 13: 13,
 14: 14,
 15: 16,
 16: 17,
 17: 18,
 18: 19,
 19: 20,
 20: 21,
 21: 22,
 22: 23,
 23: 24,
 24: 25,
 25: 26,
 26: 27,
 27: 28,
 28: 29,
 29: 30,
 30: 31,
 31: 32,
 32: 33,
 33: 34,
 34: 35,
 35: 36,
 36: 37,
 37: 38,
 38: 39,
 39: 40,
 40: 41,
 41: 42,
 42: 43,
 43: 45,
 # 44: 46,##-31
 44: 47,
 45: 48,
 46: 49,
 47: 50,
 48: 51,
 49: 52,
 50: 53,
 51: 54,
 52: 57,
 53: 59,
 54: 60,
 55: 61,
 56: 62,
 57: 63,
 58: 64,
 59: 65,
 60: 66,
 61: 67,
 62: 68,
 63: 69,
 64: 70,
 65: 71,
 66: 72,
 67: 73,
 # 69: 74,#-72
 68: 75,
 69: 76,
 70: 77,
 71: 78,
 72: 79,
 73: 80,
 74: 81,
 75: 82,
 76: 83,
 77: 84,
 78: 85,
 79: 86,
 80: 87,
 81: 88,
 82: 94,
 83: 95,
 84: 97,
 85: 101,
 86: 109,
 # 89: 111,#87
 87: 114,
 88: 115,
 89: 120,
 90: 123,
 91: 126,
 92: 127,
 93: 128,
 94: 129,
 95: 132,
 96: 133}
df_loc['class']=df_loc['class'].map(dic)

df_loc.to_csv('D:\\PyCharm\\PyProjects\\NN\\submission_final.txt',sep='\t',index=False,header=False)

7